# Week analysis

Goal of this notebook is analyze last 10 trading days for specified stock and determine if the daily trade was profitable or not. Notebook assumes that trade happened in the days 5-10 and days 1-5 are analyzed for futher enhancing knowledge and stock screener settings.

**For better analysis, run the notebook when market is closed**

Results of the notebook are following:
* Analysis if the trade was profitable, or if the stop loss was triggered
* Following technical analysis:



In [1]:
# general imports
import pandas as pd
import datetime as dt
# imports stockandloss
import scraper as sc
import analyzer as an

## General settings

In [2]:
# stocks
sh = "AAIC"

# exit points [%]
profit = 5
loss = 2

## Trade days (day 5-10)

Analysis of days 5-10 where trade happened, assuming entry point at the open of day 5 (price taken as a first open value of that day)

In [3]:
# stock 
# data scraper settings
interval = "1m"
# end date
ed = dt.date.today()
# number of days 
d = 7

# scrape trade days data
tdata, tinfo = sc.scrapstockdata(stockHandle=sh, d=d, i=interval, ed=ed)

In [4]:
# Evaluation
# save 


Scraping https://finance.yahoo.com/
Status: started
Status: completed


## Before trade days (day 1-5)

Analysis of days 1-5 mainly aimed for enhancing stock screener settings

In [5]:
# stock 
# data scraper settings
interval = "1m"
# end date
ed = dt.date.today() - dt.timedelta(days=7)
# number of days 
d = 7

# scrape trade days data
bdata, binfo = sc.scrapstockdata(stockHandle=sh, d=d, i=interval, ed=ed)